# A - Select sites to process

## Import libraries and functions

In [ ]:
import os, sys, datetime, pandas as pd
sys.path.append("/home/gswinnen/SARSAR_Package_RenPri/code/") # path to SARSAR libraries

from issep import sarsar_admin
from os.path import join
from lecture_ini import config

## Definition of the function: sites_to_process

In [ ]:
def sites_to_process_experimental(dteDebut, dteFin):
    """
    This function determines the lists of sites (id_segment) for which the SARSAR processing chain has to be run.
    They consists of sites which are notified as "active" in the "sar_def" table 
    and for which there are enough index values available in the "sar_index_stats" table over a relevant date range (dteDebut-dteFin).

        Parameters (they are automatically read in "sarsar.ini" by the function "config")::
            dteDebut (date): date from which we check data availability for a given site (YYYY-MM-DD)
            dteFin (date): date until which we check data availability for a given site (YYYY-MM-DD)
        Return :
            sites (list): list of sites (sar_id_segment) to process
    """
     
    ## CONNECTION TO DB
    
    # Define Database connection parameters
    # NOTE: password is in ~/.pgpass

    credentials = config(section='postgresql')

    db_credentials = {
        'host': credentials['host'],
        'user': credentials['user'],
        'db' : credentials['database']
    }

    # ALWAYS prepare env at the beginning
    print('> Preparing env (DB credentials, etc)')
    sarsar_admin.prepare_env(db_credentials)
    
    conn = sarsar_admin._create_or_get_db_connection()
    cur = None
    cur2 = None
    
    
    ## SELECTION OF SITES TO PROCESS  
    listeSites_NDVI = []
    listeSites_BAI = []
    listeSites_BI = []
    listeSites_BI2 = []
    listeSites_SBI = []
    listeSites_NDWI2 = []

    try:
        import psycopg2.extras # allow communication with pgsql
        cur = conn.cursor(cursor_factory = psycopg2.extras.DictCursor)

#         # listeSites_NDVI
#         strSQL = '''select i.sar_id_segment, count(i.index_name) as ndvi 
#                     from sar_index_stats i, sar_def d 
#                     where d.id_segment = i.sar_id_segment 
#                     and d.is_active = true 
#                     and index_name = 'NDVI' and acq_date between '{0}' and '{1}' 
#                     group by sar_id_segment;'''.format(dteDebut, dteFin)
        
#         cur.execute(strSQL)
#         result = cur.fetchall()
        
#         for row in result:
#             if row['ndvi'] > 0:
#                 listeSites_NDVI.append(row['sar_id_segment'])

                
#         # DEBUG:
#         print('DEBUG: listeSites_NDVI =', len(listeSites_NDVI))
#         print(listeSites_NDVI)
#         print('-----')
        
        
#         # listeSites_BAI
#         strSQL = '''select i.sar_id_segment, count(i.index_name) as bai 
#                     from sar_index_stats i, sar_def d 
#                     where d.id_segment = i.sar_id_segment 
#                     and d.is_active = true 
#                     and index_name = 'BAI' and acq_date between '{0}' and '{1}' 
#                     group by sar_id_segment;'''.format(dteDebut, dteFin)
        
#         cur.execute(strSQL)
#         result = cur.fetchall()
        
#         for row in result:
#             if row['bai'] > 0:
#                 listeSites_BAI.append(row['sar_id_segment'])

                
#         # DEBUG:
#         print('DEBUG: listeSites_BAI =', len(listeSites_BAI))
#         print(listeSites_BAI)
#         print('-----')
        
        
#         # only sites common to both lists are kept
#         sites = set(listeSites_NDVI) & set(listeSites_BAI)
        

#         # DEBUG:
#         print('DEBUG: sites =', len(sites))
#         print(sites)
#         print('-----')
        
        
#         # Delete lists to free up memory
#         del listeSites_NDVI
#         del listeSites_BAI
        
        
#         # listeSites_BI
#         strSQL = '''select i.sar_id_segment, count(i.index_name) as bi 
#                     from sar_index_stats i, sar_def d 
#                     where d.id_segment = i.sar_id_segment 
#                     and d.is_active = true 
#                     and index_name = 'BI' and acq_date between '{0}' and '{1}' 
#                     group by sar_id_segment;'''.format(dteDebut, dteFin)
        
#         cur.execute(strSQL)
#         result = cur.fetchall()
        
#         for row in result:
#             if row['bi'] >= 10:  # must have at least 10 values for the BI, BI2, SBI indexes (required for the computation)
#                 listeSites_BI.append(row['sar_id_segment'])
        
        
#         # DEBUG:
#         print('DEBUG: listeSites_BI =', len(listeSites_BI))
#         print(listeSites_BI)
#         print('-----')
        
#         # restricts the list of sites to items common to those in the new selection
#         sites = sites & set(listeSites_BI)        
#         del listeSites_BI

        
#         # DEBUG:
#         print('DEBUG: sites =', len(sites))
#         print(sites)
#         print('-----')
        
        
        # listeSites_BI2
        strSQL = '''select i.sar_id_segment, count(i.index_name) as bi2 
                    from sar_index_stats i, sar_def d 
                    where d.id_segment = i.sar_id_segment 
                    and d.is_active = true 
                    and index_name = 'BI2' and acq_date between '{0}' and '{1}' 
                    group by sar_id_segment;'''.format(dteDebut, dteFin)
        
        cur.execute(strSQL)
        result = cur.fetchall()
        
        for row in result:
            if row['bi2'] >= 10:  # must have at least 10 values for the BI, BI2, SBI indexes (required for the computation)
                listeSites_BI2.append(row['sar_id_segment'])
        
        
        # DEBUG:
        print('DEBUG: listeSites_BI2 =', len(listeSites_BI2))
        print(listeSites_BI2)
        print('-----')
        
        
        # restricts the list of sites to items common to those in the new selection
        sites = sites & set(listeSites_BI2)
        del listeSites_BI2
        
        
        # DEBUG:
        print('DEBUG: sites =', len(sites))
        print(sites)
        print('-----')
        
        
        # listeSites_SBI
        strSQL = '''select i.sar_id_segment, count(i.index_name) as sbi 
                    from sar_index_stats i, sar_def d 
                    where d.id_segment = i.sar_id_segment 
                    and d.is_active = true 
                    and index_name = 'SBI' and acq_date between '{0}' and '{1}' 
                    group by sar_id_segment;'''.format(dteDebut, dteFin)
        
        cur.execute(strSQL)
        result = cur.fetchall()
        
        for row in result:
            if row['sbi'] >= 10:  # must have at least 10 values for the BI, BI2, SBI indexes (required for the computation)
                listeSites_SBI.append(row['sar_id_segment'])
        
        
        # DEBUG:
        print('DEBUG: listeSites_SBI =', len(listeSites_SBI))
        print(listeSites_SBI)
        print('-----')
        
        
        # restricts the list of sites to items common to those in the new selection
        sites = sites & set(listeSites_SBI)
        del listeSites_SBI
        
        
        # DEBUG:
        print('DEBUG: sites =', len(sites))
        print(sites)
        print('-----')
        
        
        # listeSites_NDWI2
        strSQL = '''select i.sar_id_segment, count(i.index_name) as ndwi2 
                    from sar_index_stats i, sar_def d 
                    where d.id_segment = i.sar_id_segment 
                    and d.is_active = true 
                    and index_name = 'NDWI2' and acq_date between '{0}' and '{1}' 
                    group by sar_id_segment;'''.format(dteDebut, dteFin)
        
        cur.execute(strSQL)
        result = cur.fetchall()
        
        for row in result:
            if row['ndwi2'] > 0:
                listeSites_NDWI2.append(row['sar_id_segment'])
        
        
        # DEBUG:
        print('DEBUG: listeSites_NDWI2 =', len(listeSites_NDWI2))
        print(listeSites_NDWI2)
        print('-----')
        
        
        # restricts the list of sites to items common to those in the new selection
        sites = sites & set(listeSites_NDWI2)
        del listeSites_NDWI2
        
        return sites
            
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        return []
    
    finally:
        if cur is not None:
            cur.close()

    # ALWAYS release env at the end
    print('> Releasing env')
    sarsar_admin.release_env()

In [ ]:
sites = sites_to_process_experimental('2022-07-01', '2022-08-01')

print(len(sites))
print(sites)

In [ ]:
print(strSQL)

In [ ]:
from select_sites import sites_to_process

## Function call

In [ ]:
# DEBUG: Function call

dates = config(section='dates')
dteDebut = dates['deb']
dteFin = dates['fin']

sites = sites_to_process(dteDebut, dteFin)

print(len(sites))